In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
pd.options.mode.chained_assignment = None  # default='warn'
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
%matplotlib inline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
palette = ['#F5DF4D', '#939597', '#0F4C81', '#FF6F61', '#5F4B8B', '#88B04B', '#92A8D1', '#F7CAC9', '#955251',\
           '#B163A3', '#009473']
#palette.reverse()


/kaggle/input/fia-f1-19502019-data/70_Anniversary_Race_LapbyLap .csv
/kaggle/input/fia-f1-19502019-data/race_wins_1950-2020.csv
/kaggle/input/fia-f1-19502019-data/Spain2020_Q3_LapbyLap.csv
/kaggle/input/fia-f1-19502019-data/fastest_laps_1950-2020.csv
/kaggle/input/fia-f1-19502019-data/race_results_1950-2020.csv
/kaggle/input/fia-f1-19502019-data/constructors_championship_1958-2020.csv
/kaggle/input/fia-f1-19502019-data/70_Anniversary Q2_LapbyLap.csv
/kaggle/input/fia-f1-19502019-data/qualifying_times_1950-2005.csv
/kaggle/input/fia-f1-19502019-data/70_Anniversary Q3_LapbyLap.csv
/kaggle/input/fia-f1-19502019-data/Spain2020_Race_LapbyLap.csv
/kaggle/input/fia-f1-19502019-data/Monaz2020_Race_LapbyLap.csv
/kaggle/input/fia-f1-19502019-data/Spain2020_Q2_LapbyLap.csv
/kaggle/input/fia-f1-19502019-data/fastest_laps_all_drivers_all_race_1950-2020.csv
/kaggle/input/fia-f1-19502019-data/qualifying_times_2006-2020.csv
/kaggle/input/fia-f1-19502019-data/Spain2020_Q1_LapbyLap.csv
/kaggle/input/fia

In [2]:
cons = pd.read_csv('/kaggle/input/fia-f1-19502019-data/constructors_championship_1958-2020.csv')
cons.head()


,Unnamed: 0,Year,Position,Team,Points
0,0,1958,1,Vanwall,48
1,1,1958,2,Ferrari,40
2,2,1958,3,Cooper Climax,31
3,3,1958,4,BRM,18
4,4,1958,5,Maserati,6


In [3]:
cons.isna().any()

Unnamed: 0    False
Year          False
Position      False
Team          False
Points        False
dtype: bool

In [4]:
cons["Points"].describe()


count    655.000000
mean      62.404580
std      112.179927
min        0.000000
25%        6.500000
50%       23.000000
75%       66.000000
max      765.000000
Name: Points, dtype: float64

In [5]:
gry=cons
gry['Position'] = pd.to_numeric(gry['Position'], errors='coerce')
gry = gry[gry.Position == 1]

team_win_c= gry.groupby('Team')['Team'].count()
team_win_c = pd.DataFrame(team_win_c)
team_win_c.columns = ['Wins']
team_win_c.reset_index(level=0, inplace=True)

team_win_c.sort_values(by=['Wins'], inplace=True, ascending=False)
team_win_c = team_win_c.head(10)
team_win_c = team_win_c[::-1]

fig = px.bar(team_win_c, x='Team', y='Wins',color='Wins',width=750, height=500)
fig.update_layout(title={'text': 'Teams with The Most Constructors Championships Won','y':0.95,'x':0.5})
fig.show()

In [6]:
cons["Points"] = pd.to_numeric(cons["Points"])
cons.sort_values(by=['Points'], inplace=True, ascending=False)
dfp = cons.groupby(["Team"]).agg({"Points":"sum"}).loc[lambda d: d["Points"].gt(100)]
dfp =dfp.sort_values('Points', ascending=True)
px.bar(dfp, x=dfp.index, y="Points", color="Points")

In [7]:
gry1=cons
team_points=gry1[["Points","Team"]].groupby('Team').sum()
team_points=gry1[gry1['Position'].isin([1,2,3])]
ax= px.treemap(team_points,path=["Team"],title="Unique Driver's to stand on the Podium")
ax.show()

In [8]:
gry3=cons
gry3["Year"] = pd.to_numeric(cons["Year"])
gry3=gry3[gry3['Year'].isin([2020])]
gry3["Points"] = pd.to_numeric(gry3["Points"])
gry3.sort_values(by=['Points'], inplace=True, ascending=True)
dfp = gry3.groupby(["Team"]).agg({"Points":"sum"}).loc[lambda d: d["Points"].gt(-1)]
dfp =dfp.sort_values('Points', ascending=True)
px.bar(dfp, x=dfp.index, y="Points", color="Points")

In [9]:
driv = pd.read_csv('/kaggle/input/fia-f1-19502019-data/drivers_championship_1950-2020.csv')
driv.head()

,Unnamed: 0,Year,Position,Name,Driver Tag,Nationality,Team,Points
0,0,1950,1,Nino Farina,FAR,ITA,Alfa Romeo,30.0
1,1,1950,2,Juan Manuel Fangio,FAN,ARG,Alfa Romeo,27.0
2,2,1950,3,Luigi Fagioli,FAG,ITA,Alfa Romeo,24.0
3,3,1950,4,Louis Rosier,ROS,FRA,Talbot-Lago,13.0
4,4,1950,5,Alberto Ascari,ASC,ITA,Ferrari,11.0


In [10]:
driv.isna().any()

Unnamed: 0     False
Year           False
Position       False
Name           False
Driver Tag     False
Nationality    False
Team            True
Points         False
dtype: bool

In [11]:
driv["Points"].describe()

count    1575.000000
mean       27.820952
std        53.683241
min         0.000000
25%         3.000000
50%         8.000000
75%        29.000000
max       413.000000
Name: Points, dtype: float64

In [12]:
# Let's delete all teams that have never win championship
gry1=driv
gry1['Position'] = pd.to_numeric(gry1['Position'], errors='coerce')
gry1 = gry1[gry1.Position == 1]

driv_win_c= gry1.groupby('Name')['Name'].count()
driv_win_c = pd.DataFrame(driv_win_c)
driv_win_c.columns = ['Wins']
driv_win_c.reset_index(level=0, inplace=True)

driv_win_c.sort_values(by=['Wins'], inplace=True, ascending=False)
driv_win_c = driv_win_c.head(20)
driv_win_c = driv_win_c[::-1]

fig = px.bar(driv_win_c, x='Name', y='Wins',color='Wins',width=750, height=500)
fig.update_layout(title={'text': 'Drivers with the Most Championships Won','y':0.95,'x':0.5})
fig.show()

In [13]:

driv["Points"] = pd.to_numeric(driv["Points"])
driv.sort_values(by=['Points'], inplace=True, ascending=False)
dfp1 = driv.groupby(["Name"]).agg({"Points":"sum"}).loc[lambda d: d["Points"].gt(400)]
dfp1 =dfp1.sort_values('Points', ascending=True)
fig2 = px.bar(dfp1, x=dfp1.index, y="Points", color="Points")
fig2.show()